# 锚框

一类目标检测算法是通过锚框检测

- 提出多个称为锚框的边缘框
- 预测每一个锚框之中是否含有关注的物体
- 存在则预测真实边缘框的偏移

### 交并比

通过交并比 IoU 来计算两个锚框的相似度

- 0 表示无重叠，1表示重合


$$J(A,B)=\frac{|A\cap B|}{|A\cup B|}$$


### 赋予锚框标号

- 每一个锚框是一个训练样本
- 锚框要么表示一个背景，要么关联一个真实的边缘框
- 使用大量锚框代表着存在大量的负类样本


### 使用非极大值抑制(NMS)输出

- 每一个锚框预测一个边缘框
- NMS可以合并相似的预测
    - 选择非背景类的最大预测值
    - 去掉其他合并比大于$\theta$的框，表示已经选中
    - 重复知道所有预测要么被选中，要么被去掉

**Conclusion**

- 锚框目标检测
- 生产大量锚框，赋予标号，每个锚框作为一个样本训练
- 使用NMS除去

> 锚框的引入使得复杂度飙升

In [2]:
%matplotlib inline
import torch
from d2l import torch as d2l

torch.set_printoptions(2) # 更改打印精度

锚框的生产：

以每一个**像素为中心**生成：$宽度：w\cdot s\sqrt{r}和高度：\frac{h\cdot s}{\sqrt{r}} \qquad w,h 输入图片高宽，s锚框占图片的百分比，r锚框的高宽比$

1. 取5个s，5个r，25个组合
2. $(s_1, r_1), (s_1, r_2), \cdots ,(s_1, r_n), (s_2, r_1), (s_3, r_1), \cdots ,(s_n, r_1)$ 九个组合

In [44]:
def multibox_prior(data, sizes, ratios):
    in_height, in_width = data.shape[-2:]
    device, num_sizes, num_ratios = data.device, len(sizes), len(ratios)
    num_boxes_per_pixel = num_sizes + num_ratios - 1
    size_tensor = torch.tensor(sizes, device=device)
    ratios_tensor = torch.tensor(ratios, device=device)
    
    offset_h, offset_w = 0.5, 0.5
    steps_h, steps_w = 1.0 / in_height, 1.0 / in_width
    
    center_h = (torch.arange(in_height, device=device) + offset_h) * steps_h
    center_w = (torch.arange(in_width, device=device) + offset_w) * steps_w
    shift_y, shift_x = torch.meshgrid(center_h, center_w) # 生成网格
    shift_y, shift_x = shift_y.reshape([-1]), shift_x.reshape(-1) # 变成一行
    
    w = torch.cat((size_tensor * torch.sqrt(ratios_tensor[0]), size_tensor[0] * torch.sqrt(ratios_tensor[1:]))) \
        * in_height / in_width
    h = torch.cat((size_tensor / torch.sqrt(ratios_tensor[0]), size_tensor[0] / torch.sqrt(ratios_tensor[1:])))
    
    auchor_manipulartions = torch.stack((-w, -h, w, h)).T.repeat(in_height * in_width, 1) / 2
    out_grid = torch.stack([shift_x, shift_y, shift_x, shift_y], dim=1).repeat_interleave(num_boxes_per_pixel, dim = 0)
    out = out_grid + auchor_manipulartions
    return out.unsqueeze(0)

In [46]:
a = multibox_prior(torch.arange(36).reshape([6, 6]), sizes=[0.2, 0.4], ratios=[0.5, 1, 2])
a.shape

torch.Size([1, 144, 4])